In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-123"

In [31]:
from langchain.chat_models import init_chat_model

client = init_chat_model(
    model="gpt-4o",
    model_provider="openai"
)

In [ ]:
from langchain_core.messages import ToolMessage
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取当前位置的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市名字 e.g. 北京"
                    },
                    "country": {
                        "type": "string",
                        "description": "国家名字 e.g. 中国"
                    }
                },
                "required": [
                    "location", "country"
                ],
                "additionalProperties": False
            }
        }
    }
]

def get_weather(location: str, country: str) -> str:
    """获取当前位置的天气信息

    Args:
        location(str): 城市名字 e.g. 北京
        country(str): 国家名字 e.g. 中国
    """
    return f"{country}的{location}的天气是：晴天"

client1 = client.bind_tools(tools)

query = "深圳的天气怎么样?"
messages = [{"role": "user", "content": query}]

# 获取 function call 的结果
response = client1.invoke(messages)
# response: AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3PtrIN9pcKkQC4Aq2JTY6KA5', 'function': {'arguments': '{"location":"深圳","country":"中国"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 68, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BOSOnQUE2zGJxbfyGHjMV7vU33BOG', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4b04723b-4615-483c-9e6b-2d765d9613ef-0', tool_calls=[{'name': 'get_weather', 'args': {'location': '深圳', 'country': '中国'}, 'id': 'call_3PtrIN9pcKkQC4Aq2JTY6KA5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 19, 'total_tokens': 87, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

tool_result = get_weather(**response.tool_calls[0]["args"])
# tool_result: '中国的深圳的天气是：晴天'

messages.append(response)
messages.append(ToolMessage(content=tool_result, tool_call_id=response.tool_calls[0]["id"]))
response = client1.invoke(messages)
print(response)

content='深圳的天气是晴天。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 103, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BOSmpn1GBh9j4fXqhmLlzBOdi5tE2', 'finish_reason': 'stop', 'logprobs': None} id='run-3ed3ab75-5b28-46fb-aba8-866583e2ffb7-0' usage_metadata={'input_tokens': 103, 'output_tokens': 9, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [51]:
# 简单的方式
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
@tool
def get_weather(location: str, country: str) -> str:
    """获取当前位置的天气信息

    Args:
        location(str): 城市名字 e.g. 北京
        country(str): 国家名字 e.g. 中国
    """
    return f"{country}的{location}的天气是：晴天"

client2 = client.bind_tools([get_weather])

query = "深圳的天气怎么样?"
messages = [{"role": "user", "content": query}]

# 获取 function call 的结果
response = client2.invoke(messages)
print(response.tool_calls[0]["args"], type(response.tool_calls[0]["args"]))
tool_result = get_weather.invoke(response.tool_calls[0]["args"])
# tool_result: '中国的深圳的天气是：晴天'

messages.append(response)
messages.append(ToolMessage(content=tool_result, tool_call_id=response.tool_calls[0]["id"]))
response = client2.invoke(messages)
response

{'location': '深圳', 'country': '中国'} <class 'dict'>


AIMessage(content='深圳的天气是晴天。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 112, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f7a584cf1f', 'id': 'chatcmpl-BOSwjqh2N8wnnndHhftnzvWygukVh', 'finish_reason': 'stop', 'logprobs': None}, id='run-d16e1a74-9116-4959-a6cd-4155624a2fdf-0', usage_metadata={'input_tokens': 112, 'output_tokens': 9, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})